In [2]:
# run w aln output in both modes
import sys
sys.path.append('../src/')




#matcAFDB_tools up positions


#calc fident with cross alphabet comparison




In [28]:
import subprocess ,shlex


def runargs(args):
    '''run a command line command

    Parameters
    ----------
    args : str
        command line command
    '''

    args = shlex.split( args)
    p = subprocess.run( args )
    return p

def runFoldseek_allvall( structfolder , outfolder , foldseekpath = '../foldseek/foldseek' , mode = 2):
    '''
    run foldseek search and createtsv

    parameters
    ----------
    dbpath : str
        path to foldseek database
    outfolder : str 
        path to output folder
    maxseqs : int   
        maximum number of sequences to compare to

    '''

    args = foldseekpath + " easy-search " + structfolder + " "+ structfolder +" "+ outfolder+"aln_"+str(mode)+"_allvall.csv tmp" + " --format-output 'query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits,lddt,qaln,taln,alntmscore' --exhaustive-search --alignment-type "+ str(mode) 
    p = runargs(args)
    return outfolder+"aln_"+str(mode)+"_allvall.csv"



def read_dbfiles():
    #find positions 
    
    

In [31]:
structfolder = '../testdata/structs'
runFoldseek_allvall(structfolder , outfolder = structfolder, mode = 0)
runFoldseek_allvall(structfolder , outfolder = structfolder, mode = 2)

../testdata/structsaln_0_allvall.csv exists and will be overwritten
easy-search ../testdata/structs ../testdata/structs ../testdata/structsaln_0_allvall.csv tmp --format-output query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits,lddt,qaln,taln,alntmscore --exhaustive-search --alignment-type 0 

MMseqs Version:              	30fdcac78217579fa25d59bc271bd4f3767d3ebb
Seq. id. threshold           	0
Coverage threshold           	0
Coverage mode                	0
Max reject                   	2147483647
Max accept                   	2147483647
Add backtrace                	false
TMscore threshold            	0
TMalign hit order            	0
TMalign fast                 	1
Preload mode                 	0
Threads                      	48
Verbosity                    	3
LDDT threshold               	0
Sort by structure bit score  	1
Substitution matrix          	aa:3di.out,nucl:3di.out
Alignment mode               	3
Alignment mode               	0
E-value threshol

'../testdata/structsaln_2_allvall.csv'

In [7]:
import pandas as pd

In [40]:
alndf0 = pd.read_table('../testdata/structsaln_0_allvall.csv' , header = None)
alndf2 = pd.read_table('../testdata/structsaln_2_allvall.csv', header = None)
columns = 'query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits,lddt,qaln,taln,alntmscore'.split(',')
alndf0.columns = columns
alndf2.columns = columns

alndf0 = alndf0.set_index(['query', 'target'])
alndf2 = alndf2.set_index(['query', 'target'])


alndf = alndf0.merge(alndf2, left_index=True, right_on=['query', 'target'])
print(alndf)

                       fident_x  alnlen_x  mismatch_x  gapopen_x  qstart_x  \
query      target                                                            
Q5IS80.pdb Q5IS80.pdb     1.000       770           0          0         1   
           P53601.pdb     0.910       770          68          1         2   
           Q60495.pdb     0.915       772          61          3         2   
           P79307.pdb     0.863       770         105          0         1   
           P05067.pdb     0.836       773         121          3         1   
...                         ...       ...         ...        ...       ...   
Q29149.pdb P05067.pdb     0.846        13           2          0         1   
           Q06481.pdb     0.785        14           3          0         1   
           P12023.pdb     0.846        13           2          0         1   
           Q28053.pdb     1.000        10           0          0         1   
           P08592.pdb     0.687        16           5          0

In [43]:
#use the qaln and taln of x and y to derive the other side fident
import numpy as np
def Fident(str1,str2):
    minlen= min( (len(str1),len(str2))  )
    str1 = str1[:minlen]
    str2 = str1[:minlen]
    
    str1 = np.array(list(str1))
    str2 = np.array(list(str2))            
    return len(np.where( (str1 == str2 ) & (str1 != '-' ) & (str2 != '-')  )[0]) / len(str1)


def copyaln( aln, seq):
    seqiter = iter(seq)
    newaln = ''
    
    minlen= min( (len(aln.replace('-','')),len(seq))  )
    for i,char in enumerate(aln):
        if i > minlen-1:
            break
        if char == '-':
            newaln += '-'
        else:
            newaln+=next(seqiter)
        
    return newaln

def calc_fident_crossaln(row , verbose = False):
    qaln_0,taln_0, qaln_2, taln_2 = row.qaln_x, row.qaln_y, row.taln_x, row.taln_y
    qAA = qaln_2.replace('-','')
    q3di = qaln_0.replace('-','')
    tAA = taln_2.replace('-','')
    t3di = taln_0.replace('-','')
    if verbose == True:
        print('q', len(qAA) , len(q3di))
        print('t', len(tAA) , len(t3di) )
    #add gaps
    qAA_newgaps = copyaln(qaln_0, qAA)
    q3di_newgaps = copyaln(qaln_2, q3di)
    tAA_newgaps = copyaln(taln_0, tAA)
    t3di_newgaps = copyaln(taln_2, t3di) 
    
    if verbose == True:
        print('q3di->AA',  qaln_0, qAA_newgaps )
        print('t3di->AA', taln_0 , tAA_newgaps )
        print('qAA3di ->3di',  qaln_0, q3di_newgaps )
        print('tAA3di ->AA', taln_0 , t3di_newgaps )

    #return columns
    return [ Fident(q3di_newgaps , t3di_newgaps) , Fident(qAA_newgaps , tAA_newgaps)]



In [44]:
testrow = alndf.iloc[113]
print(testrow)
calc_fident_crossaln(testrow, verbose = True)

fident_x                                                    0.741
alnlen_x                                                       58
mismatch_x                                                     12
gapopen_x                                                       1
qstart_x                                                      636
qend_x                                                        690
tstart_x                                                        1
tend_x                                                         58
evalue_x                                                  0.03234
bits_x                                                         28
lddt_x                                                     0.7766
qaln_x          VRTETDKRQSAGYEVYHQKLV---FFADDVGSNKGAIIGLMVGGVV...
taln_x          ISEVKMDAEFRHDSGYEVHHQKLVFFAEDVGSNKGAIIGLMVGGVV...
alntmscore_x                                               0.4461
fident_y                                                    0.775
alnlen_y  

[1.0, 0.9454545454545454]

In [25]:
alndf['Fidentx'] = alndf[['qaln_x','qaln_y', 'taln_x', 'taln_y']].apply(calc_fident_crossaln , axis =1 )
print(alndf)

                       fident_x  alnlen_x  mismatch_x  gapopen_x  qstart_x  \
query      target                                                            
Q5IS80.pdb Q5IS80.pdb     1.000       770           0          0         1   
           P53601.pdb     0.910       770          68          1         2   
           Q60495.pdb     0.915       772          61          3         2   
           P79307.pdb     0.863       770         105          0         1   
           P05067.pdb     0.836       773         121          3         1   
...                         ...       ...         ...        ...       ...   
Q29149.pdb P05067.pdb     0.846        13           2          0         1   
           Q06481.pdb     0.785        14           3          0         1   
           P12023.pdb     0.846        13           2          0         1   
           Q28053.pdb     1.000        10           0          0         1   
           P08592.pdb     0.687        16           5          0